In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, train_test_split


import re
from nltk.stem import WordNetLemmatizer


import nltk
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer as detokenize
from nltk.corpus import stopwords

from textblob import TextBlob

import datetime as dt
import warnings
import string

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

# stop_words = []
stop_words = list(set(stopwords.words('english')))
warnings.filterwarnings('ignore')
punctuation = string.punctuation

In [30]:
import flask

In [31]:
fake_data = pd.read_csv("../news-classfier-model/data/archive/Fake.csv")
true_data = pd.read_csv("../news-classfier-model/data/archive/True.csv")

In [32]:
fake_data["is_authentic"] = 0
true_data["is_authentic"] = 1

In [33]:
fake_data.head()

,title,text,subject,date,is_authentic
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [34]:
true_data.head()

,title,text,subject,date,is_authentic
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [35]:
frames = [fake_data, true_data]
  
data = pd.concat(frames)

In [36]:
data.head()

,title,text,subject,date,is_authentic
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [37]:
df = shuffle(data)
df.reset_index(inplace=True, drop=True)

In [38]:
df.head()

,title,text,subject,date,is_authentic
0,U.S. welcomes decision by Saudi-led coalition ...,WASHINGTON (Reuters) - The United States on Th...,worldnews,"December 21, 2017",1
1,NEW EVIDENCE PROVES Hillary Manipulated Senate...,,politics,"Sep 6, 2016",0
2,Trump's Chicago rally called off for safety re...,CHICAGO (Reuters) - U.S. presidential candidat...,politicsNews,"March 12, 2016",1
3,Here’s What Happened When Black Protesters Di...,When the Bundy Moron Militia decided to illega...,News,"January 16, 2016",0
4,India calls Rohingya refugees 'threat to natio...,NEW DELHI (Reuters) - The Indian government on...,worldnews,"September 14, 2017",1


In [39]:
df.isnull().sum()

title           0
text            0
subject         0
date            0
is_authentic    0
dtype: int64

In [40]:
def text_process(df):
    """
    Extracts some text information from both article title and content
    
    Input: 
        df (dataframe like object): This is a data read in using pandas
    
    Output: 
        df (datafame like object): This is a preprocessed data with some new columns extracted from the
            article title and content
    """
    df["title_len"] = df['title'].apply(len)  # getting the length of the title including space
    df["text_len"] = df['text'].apply(len)  # getting the length of the title including space
    df['char_count'] = df['text'].apply(len)
    df['word_count'] = df['text'].apply(lambda x: len(x.split()))  # counting the words in text excluding spaces
    df['word_density'] = df['char_count'] / (df['word_count']+1)  # average length of word used in each text
    df['punctuation_count'] = df['text'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation))) 
    df['title_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))  #total number of proper case (title) words in the content
    df['upper_case_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()])) # total number of upper count words in the content
    df['stopword_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.lower() in stop_words]))  # total number of stopwords in the content
    
    return df

df = text_process(df)

In [42]:
def count_POS(text, pos):
    """
    This methos checks if first word in sentence is a verb or not

    PARAMETERS 
        text (str): Text to be processed   
    RETURN
        Returns (bool): a boolean to confirm if first word in sentence is a verb or not
    """
    pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
    }

    word_list = nltk.word_tokenize(text)
    count = 0
    pos_tags = nltk.pos_tag(word_list)  #obtaing the pos taggings
    for tag in pos_tags:
        
        #selecting the pos tag of the first item in the sentence
        _, first_tag = tag

        #checking if the first word in the senntece has a pos tagging of VERB
        if first_tag in pos_dic[pos]:
            count += 1
    return count

In [43]:
def reduceText(text, num_sent = 4):
    """
    This function reduces the volume of the text content by selectin a number of sentence.
    
    Input:
        text (str): the text content which we want to reduce
        num_sent (int): the number of sentence we may want to select
        
    Output:
        text (str): a sample of the text that is be selected
    """
    content = detokenize().detokenize(nltk.sent_tokenize(text)[:num_sent])
    return content

In [45]:
df['reduced_text'] = df['text'].apply(lambda text: reduceText(text))

In [49]:
def extract_pos(df):
    """
    This function extracts some part of speech and saves their counts in a columns
    
    Input:
        df (dataframe): The dataframe object
    
    Output:
        df (dataframe): The extended dataframe with the new columns
    """
    
    df['noun_count'] = df['reduced_text'].apply(lambda x: count_POS(x, 'noun'))
    df['verb_count'] = df['reduced_text'].apply(lambda x: count_POS(x, 'verb'))
    df['adj_count'] = df['reduced_text'].apply(lambda x: count_POS(x, 'adj'))
    df['adv_count'] = df['reduced_text'].apply(lambda x: count_POS(x, 'adv'))
    df['pron_count'] = df['reduced_text'].apply(lambda x: count_POS(x, 'pron'))
    
    return df

df = extract_pos(df)

In [ ]:
# dropping unused colums

df.drop(['text', 'author', 'char_count', 'id'], axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7,5))
ax = sns.countplot(x='label', data=df, ax = axes,);
plt.title("Distribution of Fake News by counts")

for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format(height/float(len(df))*100)+'%',
            ha="center")